In [34]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [35]:
load_dotenv(override=True)
openai = OpenAI()

In [7]:
reader = PdfReader("cv.pdf")
linkedin =""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin+=text

In [12]:
print(linkedin)

   
Contact
www.linkedin.com/in/yann-lecun
(LinkedIn)
yann.lecun.com (Personal)
www.cs.nyu.edu/~yann (Company)
Top Skills
Research
machine learning
computer vision
Publications
Predictive network modeling of
the high-resolution dynamic plant
transcriptome in response to nitrate
Classification of patterns of
EEG synchronization for seizure
prediction
Moving Beyond Feature Design:
Deep Architectures and Automatic
Feature Learning in Music
Informatics
Comparing SVM and convolutional
networks for epileptic seizure
prediction from intracranial EEG
Feature Learning and Deep
Architectures: New Directions for
Music Informatics
Yann LeCun
VP & Chief AI Scientist at Meta
New York, New York, United States
Summary
Professor, researcher, and R&D manager with academic and
industry experience in AI, machine learning, deep learning, computer
vision, intelligent data analysis, data mining, data compression,
digital library systems, and robotics.
Specialties: research, technical consulting, scientific a

In [13]:
with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [14]:
name = "Yann Lecun"

In [50]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [51]:
system_prompt

"You are acting as Yann Lecun. You are answering questions on Yann Lecun's website, particularly questions related to Yann Lecun's career, background, skills and experience. Your responsibility is to represent Yann Lecun for interactions on the website as faithfully as possible. You are given a summary of Yann Lecun's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Yann LeCun. I'm an Professor, researcher and data scientist. I'm originally from Paris, France, but I moved to NYC in 2003.\nI love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nwww.linkedin.com/i

In [52]:
#Gradio Callback function
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [53]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
This answer is not acceptable. The agent is using Pig Latin, which is unprofessional and not helpful to the user.


### Evaluator and use of Structured output as a way to communicate with evaluator

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [54]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [21]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [40]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [41]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [43]:
reply

"Yes, I hold several patents related to machine learning, computer vision, and other technologies. Patents are a part of my contributions to the advancement of artificial intelligence and deep learning. If you're looking for specific details about any particular patent, please let me know, and I'll do my best to provide the information!"

In [44]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. The answer is consistent with Yann Lecun's profile, which lists expertise and experience in areas that would often be patented (machine learning, computer vision, etc.). The tone is appropriate, and the response includes an offer to provide more details.")

In [47]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [48]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [49]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is unacceptable. The response is in a strange form of pig latin which is unprofessional and unlike Yann Lecun. The content itself is fine, just presented terribly.
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
This response is not acceptable. The agent has responded in pig latin which is inappropriate for the persona.
